In [1]:
import torch.nn as nn
import torch.nn.init as init

M = 0.0001
sdev = 0.0005

def weight_init(m):
    if isinstance(m, nn.Conv1d):
        init.normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.Conv2d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.Conv3d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.ConvTranspose1d):
        init.normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.ConvTranspose2d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.ConvTranspose3d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.BatchNorm1d):
        init.normal_(m.weight.data, mean=M, std=sdev)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm2d):
        init.normal_(m.weight.data, mean=M, std=sdev)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm3d):
        init.normal_(m.weight.data, mean=M, std=sdev)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        init.xavier_normal_(m.weight.data)
        init.normal_(m.bias.data)
    elif isinstance(m, nn.LSTM):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)
    elif isinstance(m, nn.LSTMCell):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)
    elif isinstance(m, nn.GRU):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)
    elif isinstance(m, nn.GRUCell):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)

In [2]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import torch.utils.data
import torch.nn.functional as F
from tqdm import tqdm
import pandas as pd
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.init as init

class ann(nn.Module):
    def __init__(self):
        super(ann,self).__init__()
        self.model = nn.Sequential(
            nn.Linear(22,10),
            nn.ReLU(),
            nn.Linear(10,1),
        )
    def forward(self, x):
        return self.model(x)

class dataset(Dataset):
    def __init__(self, train_x, train_y):
        self.x = train_x
        self.y = train_y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        return self.x[index], self.y[index]

df = pd.read_csv('dataset.csv', delimiter=',', decimal = ',')
df = df.dropna()

Y = df['% Silica Concentrate']
X = df.drop(['% Silica Concentrate', 'date'], axis = 1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 42)

X_train = X_train.values
X_test = X_test.values
Y_train = Y_train.values
Y_test = Y_test.values

train_dataset = dataset(X_train, Y_train)
test_dataset = dataset(X_test, Y_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 100, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 100, shuffle = True)

In [3]:
model = ann()
model.apply(weight_init)
model = model.double()
if torch.cuda.is_available():
    model = model.cuda()
train_losses = []
val_losses = []

In [4]:
learning_rate = 0.000000001
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, weight_decay = 10e-5)
criterion = nn.MSELoss()

In [5]:
from tqdm import tqdm_notebook as tqdm

In [6]:
num_epochs = 20

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 100, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 100, shuffle = True)

for epoch in range(1,num_epochs,1):
    print("Epoch: ",epoch)
    running_loss = 0.0
    for i, dat in enumerate(tqdm(train_loader), 0):
        inputs, labels = dat
        inputs = inputs.resize_(100,1,22)
        labels = labels.resize_(100,1,1)
        if torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss = running_loss + loss.item()
    print("T: ", running_loss/len(X_train))
    train_losses.append(running_loss/len(X_train))
    running_loss = 0.0
    for i, dat in enumerate(test_loader, 0):
        inputs, labels = dat
        inputs = inputs.resize_(100,1,22)
        labels = labels.resize_(100,1,1)
        if torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()
        outputs = model(inputs)
        running_loss = running_loss + loss.item()
    print("V: ", running_loss/len(X_test))
    val_losses.append(running_loss/len(X_test))

Epoch:  1



T:  74.52309819362395
V:  23.508864662429737
Epoch:  2



T:  24.944801115415498
V:  19.044135567324847
Epoch:  3



T:  19.365111207497467
V:  16.488439479766896
Epoch:  4



T:  16.59510815411756
V:  16.30252933132236
Epoch:  5


...


T:  5.929918969785873
V:  6.104522511462302
Epoch:  16



T:  5.551349172766688
V:  6.219281192378214
Epoch:  17



T:  5.208451074924307
V:  3.8523929572735893
Epoch:  18



T:  4.8974261315839485
V:  4.448106560366884
Epoch:  19



T:  4.61431766238254
V:  5.216893267341873


In [66]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 1, shuffle = True)

Y_pred = []

for i, dat in enumerate(tqdm(test_loader), 0):
        inputs, labels = dat
        inputs = inputs.resize_(1,1,22)
        labels = labels.resize_(1,1,1)
        if torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()
        outputs = model(inputs)
        Y_pred.append(outputs[0][0].cpu().detach().numpy())

In [67]:
from sklearn.metrics import mean_squared_error
error = mean_squared_error(Y_test, Y_pred)
print(error)

2.3947273153864583


In [71]:
n = len(Y_test)

accuracy = 0.0

for i in range(n):
    if(abs(Y_test[i]-Y_pred[i]) <= 0.1 * Y_test[i]):
        accuracy = accuracy + 1

print(accuracy/n)

0.12088900540599179
